In [4]:
# 쥬피터노트북 셀 스타일 조절 
from IPython.core.display import display, HTML
display(HTML("<style>.container { font-weight: bold !important; }</style>"))
display(HTML("<style>.container { width: 100% !important; }</style>"))

# 경고 메세지 숨기기 
import warnings
warnings.filterwarnings(action='ignore')

import requests
from bs4 import BeautifulSoup
import pandas as pd

C:\Users\pusan\AppData\Local\Temp\ipykernel_9316\1233986334.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


### 기상청 전국날씨 RSS
- RSS(Really Simple Syndication, Rich Site Summary)란
- 블로그처럼 컨텐츠 업데이트가 자주 일어나는 웹사이트에서,
업데이트된 정보를 쉽게 구독자들에게 제공하기 위해 XML을 기초로 만들어진 데이터 형식
### 시간별 예보
- https://www.weather.go.kr/w/index.do 하단 [RSS] 메뉴 클릭

- [동네예보 > 시간별 예보] 에서 지역 지정 후 [RSS] 클릭
- 예시 ) 부산광역시 수영구 광안1동

- 주소 복사 : http://www.kma.go.kr/wid/queryDFSRSS.jsp?zone=2650076000

- XML 가이드 => https://www.kma.go.kr/images/weather/lifenindustry/timeseries_XML.pdf

In [10]:
# 1) 웹페이지 데이타 (RSS) 요청 => xml 텍스트 => 뷰티풀숩객체

# 기상청 날씨 누리 https://www.weather.go.kr/w/index.do
# [RSS] 클릭 후 지역 선택 
# 예시 - 부산광역시 수영구 광안1동 선택 후 [RSS] 클릭 
# 주소 복사 

url = 'https://www.kma.go.kr/wid/queryDFSRSS.jsp?zone=2650076000'
res = requests.get(url)
xml_str = res.text
# xml_str
soup = BeautifulSoup(xml_str, 'xml')
soup

<?xml version="1.0" encoding="utf-8"?>
<rss version="2.0">
<channel>
<title>기상청 동네예보 웹서비스 - 부산광역시 수영구 광안제1동 도표예보</title>
<link>http://www.kma.go.kr/weather/main.jsp</link>
<description>동네예보 웹서비스</description>
<language>ko</language>
<generator>동네예보</generator>
<pubDate>2022년 06월 08일 (수)요일 14:00</pubDate>
<item>
<author>기상청</author>
<category>부산광역시 수영구 광안제1동</category>
<title>동네예보(도표) : 부산광역시 수영구 광안제1동 [X=99,Y=75]</title><link>http://www.kma.go.kr/weather/forecast/timeseries.jsp?searchType=INTEREST&amp;dongCode=2650076000</link>
<guid>http://www.kma.go.kr/weather/forecast/timeseries.jsp?searchType=INTEREST&amp;dongCode=2650076000</guid>
<description>
<header>
<tm>202206081400</tm>
<ts>4</ts>
<x>99</x>
<y>75</y>
</header>
<body>
<data seq="0">
<hour>18</hour>
<day>0</day>
<temp>19.0</temp>
<tmx>-999.0</tmx>
<tmn>-999.0</tmn>
<sky>1</sky>
<pty>0</pty>
<wfKor>맑음</wfKor>
<wfEn>Clear</wfEn>
<pop>0</pop>
<r12>0.0</r12>
<s12>0.0</s12>
<ws>6.9</ws>
<wd>1</wd>
<wdKor>북동</wdKor>
<wdEn>NE</wdEn>
<

In [13]:
# 2) 데이타 리스트 생성
target_list = soup.find_all('data')
len(target_list)

19

In [18]:
target_list[0]

<data seq="0">
<hour>18</hour>
<day>0</day>
<temp>19.0</temp>
<tmx>-999.0</tmx>
<tmn>-999.0</tmn>
<sky>1</sky>
<pty>0</pty>
<wfKor>맑음</wfKor>
<wfEn>Clear</wfEn>
<pop>0</pop>
<r12>0.0</r12>
<s12>0.0</s12>
<ws>6.9</ws>
<wd>1</wd>
<wdKor>북동</wdKor>
<wdEn>NE</wdEn>
<reh>65</reh>
<r06>0.0</r06>
<s06>0.0</s06>
</data>

In [65]:
# 3) 첫번째 요소만 정보 추출
# '날짜', '시간','기온', '하늘상태','날씨상태','강수상태','강수률','풍속(m/s)'
# 날짜  시간  기온  하늘상태    강수확률  풍속(m/s)   습도
# 날짜 day
# 0 오늘 / 1 내일 / 2 모레
day_dict = {'0':'오늘', '1':'내일', '2':'모레'}
# day = target_list[0].day.text
print(target_list[0].day)
print(target_list[0].day.text)

# day 태그의 텍스트값이 day_dict 딕셔너리의 키값으로 삽입 => 키에 해당하는 값 출력
print(day_dict[target_list[0].day.text])
day = day_dict[target_list[0].day.text]

hour = target_list[0].hour.text
temp = target_list[0].temp.text

# 하늘상태
sky_dict = {'1':'맑음', '2':'구름조금', '3':'구름많음', '4':'흐림'}
sky = sky_dict[target_list[0].sky.text]

# 강수확률
pop = target_list[0].pop.text

ws = target_list[0].ws.text
reh = target_list[0].reh.text

print('='*50)
print('날짜 시간 기온 하늘상태 강수확률 풍속(m/s) 습도')
print(f'{day:<3} {hour:<3} {temp:<5} {sky:<5} {pop} {ws:^4} {reh}')
print(day, hour, temp, sky, pop, ws, reh)

<day>0</day>
0
오늘
날짜 시간 기온 하늘상태 강수확률 풍속(m/s) 습도
오늘  18  19.0  맑음    0 6.9  65
오늘 18 19.0 맑음 0 6.9 65


In [67]:
# 4) 2차원 리스트화 => 데이타프레임화
# 날짜  시간  기온  하늘상태  강수상태  강수확률  풍속(m/s)   습도
weather_list=[]
day_dict = {'0':'오늘', '1':'내일', '2':'모레'}
sky_dict = {'1':'맑음', '2':'구름조금', '3':'구름많음', '4':'흐림'}
for data in target_list:
    day = day_dict[data.day.text]
    hour = data.hour.text
    temp = data.temp.text
    sky = sky_dict[data.sky.text]
    pop = data.pop.text
    # 실수로 변경하고 소숫점 2자리에서 반올림
    ws = round(float(data.ws.text),2) 
    reh = data.reh.text
    weather_list.append([day, hour, temp, sky, pop, ws, reh])
# df = pd.DataFrame(weather_list, columns=['날짜','시간','기온','하늘상태','강수확률','풍속(m/s),'습도'])
df = pd.DataFrame(weather_list, 
                  columns=[ '날짜','시간', '기온',  '하늘상태',  '강수확률','풍속(m/s)', '습도' ])
df

,날짜,시간,기온,하늘상태,강수확률,풍속(m/s),습도
0,오늘,18,19.0,맑음,0,6.9,65
1,오늘,21,17.0,맑음,0,5.5,70
2,오늘,24,17.0,맑음,0,4.7,75
3,내일,3,17.0,맑음,0,4.1,75
4,내일,6,17.0,맑음,0,4.6,75
5,내일,9,20.0,맑음,0,5.0,70
6,내일,12,21.0,맑음,0,4.9,70
7,내일,15,21.0,맑음,0,4.6,70
8,내일,18,20.0,맑음,0,3.8,75
9,내일,21,19.0,맑음,0,2.6,80


### 중기 예보

- https://www.weather.go.kr/w/index.do 하단 [RSS] 메뉴 클릭
- [동네예보 > 중기 예보] 에서 해당 지역의 [RSS] 클릭
-  주소 복사 : 경상북도 예시 <br>
: http://www.kma.go.kr/weather/forecast/mid-term-rss3.jsp?stnId=143
- XML 가이드 => https://www.weather.go.kr/w/resources/pdf/midtermforecast_rss.pdf

In [5]:
# 1) 웹페이지 데이타 (RSS) 요청 => xml 텍스트 => 뷰티풀숩객체

# 기상청 날씨 누리 https://www.weather.go.kr/w/index.do
# [RSS] 클릭 후 지역 선택 
# 예시 - 부산광역시 수영구 광안1동 선택 후 [RSS] 클릭 
# 주소 복사 

url = 'https://www.kma.go.kr/weather/forecast/mid-term-rss3.jsp?stnId=143'
res = requests.get(url)
xml_str = res.text
# xml_str
soup = BeautifulSoup(xml_str, 'xml')
soup

<?xml version="1.0" encoding="utf-8"?>
<rss version="2.0">
<channel>
<title>기상청 육상 중기예보</title>
<link>http://www.kma.go.kr/weather/forecast/mid-term_06.jsp</link>
<description>기상청 날씨 웹서비스</description>
<language>ko</language>
<generator>기상청</generator>
<pubDate>2022년 06월 08일 (수)요일 06:00</pubDate>
<item>
<author>기상청</author>
<category>육상중기예보</category>
<title>경상남,북도 육상 중기예보 - 2022년 06월 08일 (수)요일 06:00 발표</title>
<link>http://www.kma.go.kr/weather/forecast/mid-term_06.jsp</link>
<guid>http://www.kma.go.kr/weather/forecast/mid-term_06.jsp</guid>
<description>
<header>
<title>경상남,북도 육상중기예보</title>
<tm>202206080600</tm>
<wf>○ (강수) 14일(화)~15일(수) 오전에 비가 오겠습니다.&lt;br /&gt;○ (기온) 아침 기온은 14~21도, 낮 기온은 23~29도로 어제(7일, 아침최저기온 13~17도, 낮최고기온 19~22도)보다 높겠습니다.&lt;br /&gt;○ (해상) 남해동부해상은 14일(화)에,동해남부해상은 14일(화) 오후~15일(수) 오전에 물결이 1.0~3.0m로 높게 일겠고, 그 밖의 날은 1.0~2.0m로 일겠습니다.&lt;br /&gt;○ (주말전망) 11일(토)~12일(일)은 구름많겠습니다. 아침 기온은 14~19도, 낮 기온은 23~28도가 되겠습니다.&lt;br /&gt;&lt;br /&gt;※ 14일(화)~15일(수)은 우리나라 주변 기압계의 변동에

In [6]:
# 3) 지역 리스트 추출
target_list = soup.find_all('location')
target_list[0]

<location wl_ver="3">
<province>대구ㆍ경상북도</province>
<city>대구</city>
<data>
<mode>A02</mode>
<tmEf>2022-06-11 00:00</tmEf>
<wf>구름많음</wf>
<tmn>17</tmn>
<tmx>27</tmx>
<reliability/>
<rnSt>30</rnSt>
</data>
<data>
<mode>A02</mode>
<tmEf>2022-06-11 12:00</tmEf>
<wf>구름많음</wf>
<tmn>17</tmn>
<tmx>27</tmx>
<reliability/>
<rnSt>20</rnSt>
</data>
<data>
<mode>A02</mode>
<tmEf>2022-06-12 00:00</tmEf>
<wf>구름많음</wf>
<tmn>17</tmn>
<tmx>28</tmx>
<reliability/>
<rnSt>20</rnSt>
</data>
<data>
<mode>A02</mode>
<tmEf>2022-06-12 12:00</tmEf>
<wf>구름많음</wf>
<tmn>17</tmn>
<tmx>28</tmx>
<reliability/>
<rnSt>20</rnSt>
</data>
<data>
<mode>A02</mode>
<tmEf>2022-06-13 00:00</tmEf>
<wf>구름많음</wf>
<tmn>17</tmn>
<tmx>27</tmx>
<reliability/>
<rnSt>30</rnSt>
</data>
<data>
<mode>A02</mode>
<tmEf>2022-06-13 12:00</tmEf>
<wf>구름많음</wf>
<tmn>17</tmn>
<tmx>27</tmx>
<reliability/>
<rnSt>30</rnSt>
</data>
<data>
<mode>A02</mode>
<tmEf>2022-06-14 00:00</tmEf>
<wf>흐림</wf>
<tmn>17</tmn>
<tmx>25</tmx>
<reliability/>
<rnSt>40</rnSt

In [16]:
# 4) 첫 지역 (대구) 요소 추출
province = target_list[0].province.text
city = target_list[0].city.text

print(province)
print(city)
print(len(target_data_list))

# 날씨 데이터만 리스트로 추출
target_data_list = target_list[0].find_all('data')

# 날씨 데이터 첫번째 요소
# data (tmEf + wf + tmn + tmx + rnSt)
tmEf = target_data_list[0].tmEf.text
wf = target_data_list[0].wf.text
tmn = target_data_list[0].tmn.text
tmx = target_data_list[0].tmx.text
rnSt = target_data_list[0].rnSt.text


print(tmEf, wf, tmn, tmx, rnSt)


대구ㆍ경상북도
대구
13
2022-06-11 00:00 구름많음 17 27 30


In [17]:
# 5) 2차원 리스트 => 데이타프레임

weather_list = []
target_list = soup.find_all('location')
for location in target_list:
    province = location.province.text
    city = location.city.text
    target_data_list = location.find_all('data')
    for data in target_data_list:
        tmEf = data.tmEf.text
        wf = data.wf.text
        tmn = data.tmn.text
        tmx = data.tmx.text
        rnSt = data.rnSt.text
        # 리스트 추가
        weather_list.append([province, city, tmEf, wf, tmn, tmx, rnSt])
df = pd.DataFrame(weather_list, columns = ['지역', '도시', '날짜', '날씨', '최저기온', '최고기온', '강수률'])
df


,지역,도시,날짜,날씨,최저기온,최고기온,강수률
0,대구ㆍ경상북도,대구,2022-06-11 00:00,구름많음,17,27,30
1,대구ㆍ경상북도,대구,2022-06-11 12:00,구름많음,17,27,20
2,대구ㆍ경상북도,대구,2022-06-12 00:00,구름많음,17,28,20
3,대구ㆍ경상북도,대구,2022-06-12 12:00,구름많음,17,28,20
4,대구ㆍ경상북도,대구,2022-06-13 00:00,구름많음,17,27,30
...,...,...,...,...,...,...,...
320,대구ㆍ경상북도,칠곡,2022-06-15 00:00,흐리고 비,17,28,60
321,대구ㆍ경상북도,칠곡,2022-06-15 12:00,구름많음,17,28,30
322,대구ㆍ경상북도,칠곡,2022-06-16 00:00,맑음,17,29,20
323,대구ㆍ경상북도,칠곡,2022-06-17 00:00,맑음,18,31,20


In [18]:
# 도시만 출력
soup.find_all('city')

[<city>대구</city>,
 <city>포항</city>,
 <city>안동</city>,
 <city>경주</city>,
 <city>울진</city>,
 <city>울릉도</city>,
 <city>독도</city>,
 <city>영덕</city>,
 <city>문경</city>,
 <city>상주</city>,
 <city>예천</city>,
 <city>영주</city>,
 <city>봉화</city>,
 <city>영양</city>,
 <city>의성</city>,
 <city>청송</city>,
 <city>김천</city>,
 <city>구미</city>,
 <city>군위</city>,
 <city>고령</city>,
 <city>성주</city>,
 <city>영천</city>,
 <city>경산</city>,
 <city>청도</city>,
 <city>칠곡</city>]

In [19]:
df.tail(13)

,지역,도시,날짜,날씨,최저기온,최고기온,강수률
312,대구ㆍ경상북도,칠곡,2022-06-11 00:00,구름많음,18,29,30
313,대구ㆍ경상북도,칠곡,2022-06-11 12:00,구름많음,18,29,20
314,대구ㆍ경상북도,칠곡,2022-06-12 00:00,구름많음,17,30,20
315,대구ㆍ경상북도,칠곡,2022-06-12 12:00,구름많음,17,30,20
316,대구ㆍ경상북도,칠곡,2022-06-13 00:00,구름많음,17,27,30
317,대구ㆍ경상북도,칠곡,2022-06-13 12:00,구름많음,17,27,30
318,대구ㆍ경상북도,칠곡,2022-06-14 00:00,흐림,17,25,40
319,대구ㆍ경상북도,칠곡,2022-06-14 12:00,흐리고 비,17,25,60
320,대구ㆍ경상북도,칠곡,2022-06-15 00:00,흐리고 비,17,28,60
321,대구ㆍ경상북도,칠곡,2022-06-15 12:00,구름많음,17,28,30
